## zk-SNARKs (Non-interactive ZKP)

In [1]:
import math
import random

from __future__ import print_function

import sys

import inspect

def print_trace(self):
    print()
    print(f'[{self.__class__.__name__}.{inspect.stack()[1].function}]')

def debug_v(expression):
    frame = sys._getframe(1)
    print(expression.lstrip('self.'), '=', repr(eval(f'vars({expression})', frame.f_globals, frame.f_locals)))

def debug(expressions):
    frame = sys._getframe(1)
    for expression in expressions.split():
        print(expression.lstrip('self.'), '=', repr(eval(expression, frame.f_globals, frame.f_locals)))

def debug_s(expressions):
    frame = sys._getframe(1)
    for expression in expressions.split(','):
        print(expression.strip().lstrip('self.'), '=', repr(eval(expression, frame.f_globals, frame.f_locals)))

### Protocol

In [2]:
class Public:
    def __init__(self, p):
        self.p = p

class Private: 
    def __init__(self, a):
        self.a = a

class Generator:
    def __init__(self, g):
        self.g = g

    def generate(self, exp):
        return self.g ** exp % pub.p

class Hasher:
    @staticmethod
    def compute(g, y, t):
        return hash((g, y, t))  % pub.p

class Message:
    def __init__(self, t, r, y, c):
        self.t = t
        self.r = r
        self.y = y
        self.c = c

class Prover:
    
    def send(self):
        print_trace(self)
        
        y = G.generate(prv.a)
        v = random.randint(1, pub.p - 1)
        t = G.generate(v)
        c = Hasher.compute(G.g, y, t)
        r = (v - c * prv.a) % (pub.p - 1)
        
        msg = Message(t, r, y, c)
        debug_v('msg')

        return msg

class Verifier:

    def recv(self, msg):
        print_trace(self)
        debug_v('msg')
        
        self.t = msg.t
        self.r = msg.r
        self.y = msg.y
        self.c = msg.c

    def validate(self):
        print_trace(self)
        accepted = self.t == G.generate(self.r) * (self.y ** self.c) % pub.p
        debug('accepted')
        
        return accepted

pub = Public(3571)
debug_v('pub')

prv = Private(7)
debug_v('prv')

G = Generator(17)
debug_v('G')

P = Prover()
V = Verifier()

V.recv(P.send())
V.validate()

pub = {'p': 3571}
prv = {'a': 7}
G = {'g': 17}

[Prover.send]
msg = {'t': 3314, 'r': 868, 'y': 2205, 'c': 2844}

[Verifier.recv]
msg = {'t': 3314, 'r': 868, 'y': 2205, 'c': 2844}

[Verifier.validate]
accepted = True


True

### Attack

In [3]:
class Attacker:
    
    def recv(self, msg):
        print_trace(self)
        debug_v('msg')
        
        self.t = msg.t
        self.r = msg.r
        self.y = msg.y
        self.c = msg.c

    def send(self):
        print_trace(self)
        
        y = self.y
        v1 = random.randint(1, pub.p - 1)
        t1 = G.generate(v1)
        c1 = pub.p - 1
        r1 = v1
        
        msg = Message(t1, r1, y, c1)
        debug_v('msg')

        return msg

P = Prover()
A = Attacker()
V = Verifier()

A.recv(P.send())
V.recv(A.send())
V.validate()


[Prover.send]
msg = {'t': 1388, 'r': 809, 'y': 2205, 'c': 1211}

[Attacker.recv]
msg = {'t': 1388, 'r': 809, 'y': 2205, 'c': 1211}

[Attacker.send]
msg = {'t': 384, 'r': 2454, 'y': 2205, 'c': 3570}

[Verifier.recv]
msg = {'t': 384, 'r': 2454, 'y': 2205, 'c': 3570}

[Verifier.validate]
accepted = True


True